In [1]:
# Load necessary libraries
%load_ext autotime
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

time: 3.34 s (started: 2024-10-17 03:41:03 -04:00)


In [2]:
# Load API key from enivornment variable
load_dotenv(dotenv_path='key.env')
api_key=os.getenv('API_KEY')


time: 0 ns (started: 2024-10-17 03:41:07 -04:00)


In [3]:
# race variables (from ACS)
race_variable = [
        'B02001_001E',  # Estimate!!Total
        'B02001_002E',  # Estimate!!Total!!White alone
        'B02001_003E',  # Estimate!!Total!!Black or African American alone
        'B02001_004E',  # Estimate!!Total!!American Indian and Alaska Native alone
        'B02001_005E',  # Estimate!!Total!!Asian alone
        'B02001_006E',  # Estimate!!Total!!Native Hawaiian and Other Pacific Islander alone
        'B02001_007E',  # Estimate!!Total!!Some other race alone
        'B02001_008E',  # Estimate!!Total!!Two or more races
        'B02001_009E',  # Estimate!!Total!!Two or more races!!Two races including Some other race
        'B02001_010E'   # Estimate!!Total!!Two or more races!!Two races excluding Some other race, and three or more races
    ]

# Define variables
variables_url = 'https://api.census.gov/data/2009/acs/acs5/variables.html'

# Request the variables page
response = requests.get(variables_url)

# Parse the variables page to extract labels
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find_all('table')

    # Extract variable labels if the table exists
    if table:
        rows = table[0].find_all('tr')[1:] 
        variable_label_mapping = {}
        for row in rows[1:]:  # Skip the header row
            cols = row.find_all('td')
            if len(cols) >= 2:
                variable_code = cols[0].text.strip()  # Get variable code
                variable_label = cols[1].text.strip()  # Get variable label

                variable_label_mapping[variable_code] = variable_label  


time: 14.5 s (started: 2024-10-17 03:41:07 -04:00)


In [4]:
# variable_label_mapping

time: 0 ns (started: 2024-10-17 03:41:21 -04:00)


In [5]:
# the URl for the ACS 5-year data
url = f'https://api.census.gov/data/2009/acs/acs5?get=NAME,{",".join(race_variable)}&for=state:*&key={api_key}'


time: 0 ns (started: 2024-10-17 03:41:21 -04:00)


In [6]:
# Fetch the census data
data_response = requests.get(url)

if data_response.status_code ==200:
    data= data_response.json()
    columns = data[0]

    # Creating the DataFrame
    df =pd.DataFrame(data[1:],columns= columns)

    # Rename the columns using the mapping
    df.rename(columns={code: variable_label_mapping.get(code, code) for code in race_variable}, inplace =True)

    # Adjust the index
    df.index = df.index+1

    # Select only relevant columns('NAME' and race variables)
    df = df[['NAME'] + [variable_label_mapping[code] for code in race_variable]]

time: 1.16 s (started: 2024-10-17 03:41:21 -04:00)


In [7]:
# Display the data
df.head()

,NAME,Estimate!!Total,Estimate!!Total!!White alone,Estimate!!Total!!Black or African American alone,Estimate!!Total!!American Indian and Alaska Native alone,Estimate!!Total!!Asian alone,Estimate!!Total!!Native Hawaiian and Other Pacific Islander alone,Estimate!!Total!!Some other race alone,Estimate!!Total!!Two or more races,Estimate!!Total!!Two or more races!!Two races including Some other race,"Estimate!!Total!!Two or more races!!Two races excluding Some other race, and three or more races"
1,Alabama,4633360,3256941,1209938,22969,46655,2025,38342,56490,4421,52069
2,Alaska,683142,467650,25161,91939,31878,4269,11613,50632,2811,47821
3,Arizona,6324865,4906936,227282,284265,153301,11045,577781,164255,51463,112792
4,Arkansas,2838143,2228798,439355,19233,31120,2505,67135,49997,4819,45178
5,California,36308527,22258042,2249404,283031,4473292,132535,5639234,1272989,383694,889295


time: 47 ms (started: 2024-10-17 03:41:22 -04:00)


In [8]:
# Conversion of DataFrame to CSV file
df.to_csv('census_data_all_state_2009.csv', index=False)

time: 15 ms (started: 2024-10-17 03:41:23 -04:00)
